<a href="https://colab.research.google.com/github/BARATZL/march-madness-supML/blob/main/NCAAMB_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting the Outcome of March Madness Basketball Games

I love watching college basketball, particularly in March. My alma mater has never been in the "big dance", but the tournament has nevertheless always been very entertaining to me.

However, I do not know much about the sport itself. When it comes time to join in the office bracket pool, my bracket's success largely hinges on my luck, or the rushed google searches made just before filling out my predictions.

The above is the main motivation behind this project. Using Machine Learning concepts I have taken in during the fall semester, can I improve upon my bracket predictions from last year (where I more or less guessed)?

## Defining success




A simple way to define success of my model is to perform better than my predictions last year. I correctly guessed 42 out of 64 of the games, about 65% in total.

That sounds pretty decent for guessing, but my predicitions got worse after round 1 and 2 of the bracket. Equally weighting my predictions by each round, my accuracy looks something like this:



---



$\frac{1}{6}(R1 acc.+ R2 acc.+ R3acc...)$

Or,

$\frac{1}{6} (\frac{24}{32} + \frac{11}{16} + \frac{3}{8} + \frac{2}{4} + \frac{2}{2} + \frac{0}{1})$ = ~.55



---



So the standards I will initially aim for is a total accuracy higher than 65%, with an average accuracy across rounds higher than 55%.

# Data Sourcing and Formatting

There are two methods I have thought of that can be appropriately formatted for a model. The tabular data should be organized as follows:

Game | Team 1 | Team 1 Season Stat 1 | ... | Team 2 | Team 2 Season Stat 1 | ... | Team 1 Score | Team 2 Score | Team 1 Win (0 for no, 1 for yes)
----|----|----|----|----|----|----|----|----|---|
Purdue v. UConn | Purdue | x | ... | UConn | y | ... | 60 | 75 | 0
UConn v. Alabama | UConn | y | ... | Alabama | z | ... | 86 | 72 | 1
...|...|...|...|...|...|...|...|...|...

With this format, our model can either:

**1**. Predict the values that Team 1 and 2 score, with a subsequent function that confirms the outcome prediction the model is making.

**2**. Predict whether or not Team 1 wins.

We can assess both, but first we need to assemble our data. First, we need to create a table of season statistics.

Assembling this data across all years will be difficult, but should be possible through extracting data from [Sports Reference](https://www.sports-reference.com/cbb/). The below code begins this process.

In [1]:
from bs4 import BeautifulSoup, Comment
import numpy as np
import requests
import pandas as pd
url = 'https://www.sports-reference.com/cbb/seasons/men/2024-school-stats.html'
test = requests.get(url)

html_content = test.text
soup = BeautifulSoup(html_content, 'html.parser')
pretty_html = soup.prettify()
tables = soup.find_all('table')  # finding table in webpage
headers = soup.find_all('th')

In [2]:
columnheaders = []
for i in headers:
 columnheaders.append(i.text)
columnheaders = columnheaders[13:50]
ncaa2324 = pd.DataFrame(columns = columnheaders)  # setting table based on webpage table columns
ncaa2324.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   School  0 non-null      object
 1   G       0 non-null      object
 2   W       0 non-null      object
 3   L       0 non-null      object
 4   W-L%    0 non-null      object
 5   SRS     0 non-null      object
 6   SOS     0 non-null      object
 7           0 non-null      object
 8   W       0 non-null      object
 9   L       0 non-null      object
 10          0 non-null      object
 11  W       0 non-null      object
 12  L       0 non-null      object
 13          0 non-null      object
 14  W       0 non-null      object
 15  L       0 non-null      object
 16          0 non-null      object
 17  Tm.     0 non-null      object
 18  Opp.    0 non-null      object
 19          0 non-null      object
 20  MP      0 non-null      object
 21  FG      0 non-null      object
 22  FGA     0 non-null      object
 23  FG

In [3]:
rows = soup.find_all('tr')
for row in rows:
  cells = row.find_all('td')
  if cells == []:
    continue
  columndata = [col.text.strip() for col in cells]
  if "NCAA" in columndata[0]:
    ncaa2324.loc[len(ncaa2324)]=columndata  # extracting everyone who made it into the NCAA tournament.

In [4]:
ncaa2324.head(5)  # checking things here

,School,G,W,L,W-L%,SRS,SOS,,W,L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Akron NCAA,35,24,11,.686,2.77,-2.08,,13,5,...,467,642,.727,363,1278,455,197,100,394,583
1,Alabama NCAA,37,25,12,.676,20.69,11.80,,13,5,...,650,842,.772,472,1467,587,256,162,438,734
2,Arizona NCAA,36,27,9,.750,24.54,9.45,,15,5,...,605,844,.717,471,1533,665,300,133,430,590
3,Auburn NCAA,35,27,8,.771,22.46,7.66,,13,5,...,609,812,.750,393,1323,622,258,215,374,678
4,Baylor NCAA,35,24,11,.686,19.50,10.71,,11,7,...,579,791,.732,399,1229,514,236,110,421,577


This table contains some of the statistics we would like to see for our team season data when we compile games from March Madness tournaments.

However, there's an issue with this webscraping method: the statistics listed include tournament games. This is problematic because we want the model to be useful prior to the tournament takes place.

If I train a model on data partially from tournaments, there's a chance that it will negatively affect the model when it is needed before teams even have a chance to compile tournament statistics.

One way to avoid this is to make sure that I only include rate/percentage statistics.

In [5]:
ncaa2324.drop(columns=['W','L'], inplace=True)

In [6]:
columns_to_drop = ncaa2324.columns[5:9]
columns_to_drop
ncaa2324.drop(columns=columns_to_drop,inplace=True)

In [7]:
ncaa2324.drop(columns=['MP','FG','FGA','FT','FTA','3P','3PA'],inplace=True)

In [8]:
ncaa2324.iloc[:,1:] = ncaa2324.iloc[:,1:].astype(float)
column_ops = ['Tm.','Opp.','ORB','TRB','AST','STL','BLK','TOV','PF']
for col in column_ops:
  ncaa2324[col] = ncaa2324[col].astype(float)
  ncaa2324[col] = ncaa2324[col]/(ncaa2324['G'].astype(float))  # setting up per game ratios

ncaa2324.rename(columns={'Tm.':'PPG',
                         'Opp.':'PAPG',
                         'ORB':'ORBPG',
                         'TRB':'TRBPG',
                         'AST':'ASTPG',
                         'STL':'STLPG',
                         'BLK':'BLKPG',
                         'TOV':'TOVPG',
                         'PF':'PFPG'},inplace=True)

In [9]:
ncaa2324.drop(columns=['G','W-L%'],inplace=True)

Now I've eliminated most of the obvious indicators of postseason success that would not be available if we try to practically apply a model next year. Next, to create game data, and stitch these seasonal statistics onto the table with game data. I plan to pull game data again from Sports Reference.

In [10]:
# first, making sure school names are uniform with the second table. This way, there's a seamless merge.
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace('NCAA$', '', regex=True)
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace('Brigham Young$', 'BYU', regex=True)
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace('Connecticut$', 'UConn', regex=True)
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace('North Carolina$', 'UNC', regex=True)
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace("Saint Mary's (CA)", "Saint Mary's", regex=False)
ncaa2324['School'] = ncaa2324['School'].str.strip().str.replace("Saint Peter's", "St. Peter's", regex=False)
ncaa2324['Year'] = url[49:53]


In [11]:
mm2324 = pd.DataFrame(columns=['T1 Seed','T1 Name','T1 Score','T2 Seed','T2 Name','T2 Score','Round'])  # making combined table template

In [12]:
mm2324

,T1 Seed,T1 Name,T1 Score,T2 Seed,T2 Name,T2 Score,Round


In [13]:
url2 = 'https://www.sports-reference.com/cbb/postseason/men/2024-ncaa.html'
test = requests.get(url2)

html_content = test.text
soup = BeautifulSoup(html_content, 'html.parser')
comments = soup.find_all(string=lambda text: isinstance(text, Comment))
i = 0
for comment in comments:
    if "game" in comment:  # the html had <--game--> comments wherever they placed bracket games.
        next_element = comment.find_next_sibling()
        try:
          seed1 = next_element.find('span').text.strip()
        except:
          break
        name1 = next_element.find('a', href=True).text.strip()
        try:
          score1 = next_element.find_all('a',href=True)[1].text.strip()
        except:
          continue
        third_element = next_element.find_next_sibling()
        seed2 = third_element.find('span').text.strip()
        name2 = third_element.find('a', href=True).text.strip()
        score2 = third_element.find_all('a',href=True)[1].text.strip()
        i += 1
        if i < 33:
          round = 1
        elif i >= 33 and i < 49:
          round = 2
        elif i >= 49 and i < 57:
          round = 3
        elif i >= 57 and i < 61:
          round = 4
        elif i >= 61 and i < 63:
          round = 5
        elif i >= 63:
          round = 6
        mm2324.loc[len(mm2324)] = [seed1,name1,score1,seed2,name2,score2,round]
mm2324.loc[len(mm2324)] = [1,'UConn',75,1,'Purdue',60,6]  # for some reason the championship game still didn't have the score. manually inputting.

In [14]:
mm2324['Game'] = mm2324['T1 Name'] + f' ('+mm2324['T1 Seed'].astype(str)+')' ' v. ' + mm2324['T2 Name'] + ' ('+mm2324['T2 Seed'].astype(str)+')'
mm2324  # the above will be the index. Important for future so we can still determine which game the model is predicting on

,T1 Seed,T1 Name,T1 Score,T2 Seed,T2 Name,T2 Score,Round,Game
0,1,UConn,91,16,Stetson,52,1,UConn (1) v. Stetson (16)
1,8,Florida Atlantic,65,9,Northwestern,77,1,Florida Atlantic (8) v. Northwestern (9)
2,5,San Diego State,69,12,UAB,65,1,San Diego State (5) v. UAB (12)
3,4,Auburn,76,13,Yale,78,1,Auburn (4) v. Yale (13)
4,6,BYU,67,11,Duquesne,71,1,BYU (6) v. Duquesne (11)
...,...,...,...,...,...,...,...,...
58,6,Clemson,77,2,Arizona,72,4,Clemson (6) v. Arizona (2)
59,4,Alabama,89,6,Clemson,82,4,Alabama (4) v. Clemson (6)
60,1,UConn,86,4,Alabama,72,5,UConn (1) v. Alabama (4)
61,1,Purdue,63,11,NC State,50,5,Purdue (1) v. NC State (11)


Now that the game table has been created, the seasonal stats for the teams can be inserted into the table.

In [15]:
ncaa2324['School'] = ncaa2324['School'].str.strip()
mm2324['T1 Name'] = mm2324['T1 Name'].str.strip()
mm2324['T2 Name'] = mm2324['T2 Name'].str.strip()
comb2324 = pd.merge(mm2324, ncaa2324,left_on='T1 Name',right_on='School',how='left')
comb2324.rename(columns={'SRS':'T1 SRS',
                         'SOS':'T1 SOS',
                         'PPG':'T1 PPG',
                         'PAPG':'T1 PAPG',
                         'ORBPG':'T1 ORBPG',
                         'TRBPG':'T1 TRBPG',
                         'ASTPG':'T1 ASTPG',
                         'FG%':'T1 FG%',
                         '3P%':'T1 3P%',
                         'FT%':'T1 FT%',
                         'STLPG':'T1 STLPG',
                         'BLKPG':'T1 BLKPG',
                         'TOVPG':'T1 TOVPG',
                         'PFPG':'T1 PFPG'},inplace=True)
comb2324 = pd.merge(comb2324, ncaa2324, left_on='T2 Name',right_on='School',how='left')
comb2324.rename(columns={'SRS':'T2 SRS',
                         'SOS':'T2 SOS',
                         'PPG':'T2 PPG',
                         'PAPG':'T2 PAPG',
                         'ORBPG':'T2 ORBPG',
                         'TRBPG':'T2 TRBPG',
                         'ASTPG':'T2 ASTPG',
                         'FG%':'T2 FG%',
                         '3P%':'T2 3P%',
                         'FT%':'T2 FT%',
                         'STLPG':'T2 STLPG',
                         'BLKPG':'T2 BLKPG',
                         'TOVPG':'T2 TOVPG',
                         'PFPG':'T2 PFPG'}, inplace=True)
comb2324 = comb2324.drop(columns=['School_x','School_y','Year_y'])
comb2324.rename(columns={'Year_x':'Year'},inplace=True)
comb2324.head(5)

,T1 Seed,T1 Name,T1 Score,T2 Seed,T2 Name,T2 Score,Round,Game,T1 SRS,T1 SOS,...,T2 FG%,T2 3P%,T2 FT%,T2 ORBPG,T2 TRBPG,T2 ASTPG,T2 STLPG,T2 BLKPG,T2 TOVPG,T2 PFPG
0,1,UConn,91,16,Stetson,52,1,UConn (1) v. Stetson (16),26.7,8.7,...,0.463,0.365,0.766,9.685714,34.742857,13.285714,5.342857,3.085714,10.542857,14.742857
1,8,Florida Atlantic,65,9,Northwestern,77,1,Florida Atlantic (8) v. Northwestern (9),13.26,4.61,...,0.455,0.39,0.752,8.529412,31.411765,15.588235,7.029412,3.117647,8.823529,17.588235
2,5,San Diego State,69,12,UAB,65,1,San Diego State (5) v. UAB (12),14.68,8.37,...,0.449,0.327,0.746,12.885714,37.971429,13.514286,6.742857,4.542857,11.600000,15.857143
3,4,Auburn,76,13,Yale,78,1,Auburn (4) v. Yale (13),22.46,7.66,...,0.467,0.351,0.707,9.878788,36.515152,15.000000,6.181818,3.181818,9.575758,15.121212
4,6,BYU,67,11,Duquesne,71,1,BYU (6) v. Duquesne (11),19.33,7.86,...,0.436,0.34,0.72,10.513514,34.810811,13.432432,7.486486,4.324324,11.702703,17.135135


In [16]:
comb2324.set_index('Game',inplace=True)

In [17]:
combi2324 = comb2324.iloc[:,[21,6,0,1,7,8,9,10,11,12,13,14,15,16,17,18,19,3,4,20,22,23,24,25,26,27,28,29,30,31,32,33,34,2,5]]  # rearranging.
combi2324.info()  # null check

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, UConn (1) v. Stetson (16) to UConn (1) v. Purdue (1)
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      63 non-null     object 
 1   Round     63 non-null     int64  
 2   T1 Seed   63 non-null     object 
 3   T1 Name   63 non-null     object 
 4   T1 SRS    63 non-null     object 
 5   T1 SOS    63 non-null     object 
 6   T1 PPG    63 non-null     float64
 7   T1 PAPG   63 non-null     float64
 8   T1 FG%    63 non-null     object 
 9   T1 3P%    63 non-null     object 
 10  T1 FT%    63 non-null     object 
 11  T1 ORBPG  63 non-null     float64
 12  T1 TRBPG  63 non-null     float64
 13  T1 ASTPG  63 non-null     float64
 14  T1 STLPG  63 non-null     float64
 15  T1 BLKPG  63 non-null     float64
 16  T1 TOVPG  63 non-null     float64
 17  T2 Seed   63 non-null     object 
 18  T2 Name   63 non-null     object 
 19  T1 PFPG   63 non-null     float64

In [18]:
combi2324.head(1)

,Year,Round,T1 Seed,T1 Name,T1 SRS,T1 SOS,T1 PPG,T1 PAPG,T1 FG%,T1 3P%,...,T2 3P%,T2 FT%,T2 ORBPG,T2 TRBPG,T2 ASTPG,T2 STLPG,T2 BLKPG,T2 TOVPG,T1 Score,T2 Score
Game,,,,,,,,,,,,,,,,,,,,,
UConn (1) v. Stetson (16),2024,1,1,UConn,26.7,8.7,81.4,63.4,0.497,0.358,...,0.365,0.766,9.685714,34.742857,13.285714,5.342857,3.085714,10.542857,91,52


In [19]:
col_list = combi2324.columns
for col in col_list:
  try:
    combi2324.loc[:,col] = combi2324[col].astype(float)
  except:
    continue
combi2324.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, UConn (1) v. Stetson (16) to UConn (1) v. Purdue (1)
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      63 non-null     object 
 1   Round     63 non-null     int64  
 2   T1 Seed   63 non-null     object 
 3   T1 Name   63 non-null     object 
 4   T1 SRS    63 non-null     object 
 5   T1 SOS    63 non-null     object 
 6   T1 PPG    63 non-null     float64
 7   T1 PAPG   63 non-null     float64
 8   T1 FG%    63 non-null     object 
 9   T1 3P%    63 non-null     object 
 10  T1 FT%    63 non-null     object 
 11  T1 ORBPG  63 non-null     float64
 12  T1 TRBPG  63 non-null     float64
 13  T1 ASTPG  63 non-null     float64
 14  T1 STLPG  63 non-null     float64
 15  T1 BLKPG  63 non-null     float64
 16  T1 TOVPG  63 non-null     float64
 17  T2 Seed   63 non-null     object 
 18  T2 Name   63 non-null     object 
 19  T1 PFPG   63 non-null     float64

This is the ideal table setup we're looking for. Before iterating on the past two-ish decades of March Madness basketball, I'd like to see how the model does with a train-test set of 2024.

## ML Test Run

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import MultiTaskLasso

X = combi2324.drop(columns=['T1 Name', 'T2 Name', 'T1 Score', 'T2 Score'])
y = combi2324[['T1 Score', 'T2 Score']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
preprocessing = Pipeline([('scaler', StandardScaler())])
lasso_pipe = Pipeline([('preprocessing', preprocessing),('lasso',MultiTaskLasso(alpha=0.1,random_state=1))])
lasso_pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('scaler', StandardScaler())])),
                ('lasso', MultiTaskLasso(alpha=0.1, random_state=1))])

In [21]:
predictions = lasso_pipe.predict(X_test)

In [22]:
yt_copy = y_test.copy()
yt_copy['T1 Win'] = yt_copy['T1 Score'] > yt_copy['T2 Score']
yt_copy['Prediction 1'] = [row[0] for row in predictions]
yt_copy['Prediction 2'] = [row[1] for row in predictions]
yt_copy['Predict T1 Win'] = yt_copy['Prediction 1'] > yt_copy['Prediction 2']
yt_copy['Prediction Correct?'] = yt_copy['Predict T1 Win'] == yt_copy['T1 Win']
yt_copy

,T1 Score,T2 Score,T1 Win,Prediction 1,Prediction 2,Predict T1 Win,Prediction Correct?
Game,,,,,,,
Gonzaga (5) v. Kansas (4),89.0,68.0,True,81.296784,44.080267,True,True
James Madison (12) v. Duke (4),55.0,93.0,False,64.255709,90.575426,False,True
Dayton (7) v. Nevada (10),63.0,60.0,True,67.724757,63.622544,True,True
Purdue (1) v. Gonzaga (5),80.0,68.0,True,95.143933,86.143927,True,True
Purdue (1) v. NC State (11),63.0,50.0,True,77.261043,70.591642,True,True
San Diego State (5) v. UAB (12),69.0,65.0,True,81.162500,68.162860,True,True
Texas (7) v. Colorado State (10),56.0,44.0,True,76.516751,60.646581,True,True
Houston (1) v. Duke (4),51.0,54.0,False,69.094540,53.520000,True,False
Colorado (10) v. Marquette (2),77.0,81.0,False,54.756565,54.230359,True,False


With a really small sample size, the model predicts 9 outcomes correctly out of 13 (69% accuracy). This is surprisingly good to me! But there are a few caveats:

- The model does miss notable upsets of the test set. Duke v.s. NC State, Houston v.s. Duke, and Kentucky v.s. Oakland, with the last game's predicted scores being way off the mark.
- Even though it predicts the outcome correctly on a decent portion of the games, the predicted scores are often not close to the true score.
- This is applying all different rounds of tournament basketball at once, which we won't be able to do when using it practically. When tested with more data later, we will test it mirroring the process we would in March (Round 1 predictions, then Round 2, and so on).

What features does lasso view as important?


In [23]:
lasso_model = lasso_pipe.named_steps['lasso']
lasso_model.coef_

array([[  0.        ,  -1.05317525,  -7.13184179,  -9.03250214,
          3.05121236,   3.66315746,   2.96309003,  -0.7754551 ,
          1.53990252,  -0.12240027,   0.58154614,   0.30429675,
          2.83219248,   2.34320164,   2.73895439,  -2.76841718,
          0.73108352,  -0.41352975,  -5.0925761 ,  -0.17866062,
         -1.48410999,   1.47857105,   5.88799531,  -1.86842332,
          1.08246204,   5.69637841,  -1.01074265,   2.10273344,
         -0.87214437,  -0.51732475,  -0.97662911],
       [  0.        ,  -0.43875856,  -3.04039737,  -3.27514777,
         -2.56505003,   5.87624475,   5.23810706, -10.97232764,
          0.14627904,  -3.94314681,  -0.08778832,  -5.00063166,
          8.39237355,  -2.68826179,  -1.48410788,   2.53267619,
          0.50634858,  -0.68814904,  -0.14436828,  -0.66311592,
          1.3810354 ,  -0.51250934,   3.24651338,   2.2740633 ,
          4.84737613,   4.06398919,   3.37066151,  -0.42818836,
          2.35729383,  -2.148015  ,  -3.53862804]])

The two arrays above raise an issue with using any form of linear regression method for multi target regression. Because linear regression seeks to minimize error on each target, a team will be predicted to score a different value simply by virtue of being on the opposite side of the table I created. I'm not sure if this is something I want in my model.

What if I tried a Random Forest Regressor?



In [24]:
from sklearn.ensemble import RandomForestRegressor
rf_pipe = Pipeline([('preprocessing',preprocessing),('rf',RandomForestRegressor(n_estimators=100,random_state=1))])
rf_pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('scaler', StandardScaler())])),
                ('rf', RandomForestRegressor(random_state=1))])

In [25]:
rf_predictions = rf_pipe.predict(X_test)
yt_rfcopy = y_test.copy()
yt_rfcopy['T1 Win'] = yt_rfcopy['T1 Score'] > yt_rfcopy['T2 Score']
yt_rfcopy['Prediction 1'] = [row[0] for row in rf_predictions]
yt_rfcopy['Prediction 2'] = [row[1] for row in rf_predictions]
yt_rfcopy['Predict T1 Win'] = yt_rfcopy['Prediction 1'] > yt_rfcopy['Prediction 2']
yt_rfcopy['Prediction Correct?'] = yt_rfcopy['Predict T1 Win'] == yt_rfcopy['T1 Win']
yt_rfcopy

,T1 Score,T2 Score,T1 Win,Prediction 1,Prediction 2,Predict T1 Win,Prediction Correct?
Game,,,,,,,
Gonzaga (5) v. Kansas (4),89.0,68.0,True,79.14,70.82,True,True
James Madison (12) v. Duke (4),55.0,93.0,False,70.62,71.57,False,True
Dayton (7) v. Nevada (10),63.0,60.0,True,69.21,69.82,False,False
Purdue (1) v. Gonzaga (5),80.0,68.0,True,86.65,65.26,True,True
Purdue (1) v. NC State (11),63.0,50.0,True,83.15,63.46,True,True
San Diego State (5) v. UAB (12),69.0,65.0,True,75.20,72.78,True,True
Texas (7) v. Colorado State (10),56.0,44.0,True,67.80,66.72,True,True
Houston (1) v. Duke (4),51.0,54.0,False,78.11,63.85,True,False
Colorado (10) v. Marquette (2),77.0,81.0,False,70.06,68.95,True,False


In [26]:
rf_model = rf_pipe.named_steps['rf']
rf_model.feature_importances_


array([0.        , 0.00864146, 0.06262339, 0.10539343, 0.03248482,
       0.04817453, 0.09870799, 0.02880143, 0.01275978, 0.01070208,
       0.02552077, 0.01957543, 0.02024651, 0.01871674, 0.01478693,
       0.03482568, 0.02124226, 0.02638043, 0.04146786, 0.03286671,
       0.01435417, 0.10227344, 0.04608148, 0.02153138, 0.01157179,
       0.01718818, 0.03041166, 0.0420202 , 0.01077431, 0.02381103,
       0.01606413])

Random Forest does slightly worse, but the lack of data should be noted. I think it's time to expand the data we are training on. We'll use code implemented earlier as functions to iterate on lots of Sports Reference pages.

In [27]:
def get_season_stats(url_name):  # only works with Sports Reference!!
  test = requests.get(url_name)

  html_content = test.text  # setting up the html
  soup = BeautifulSoup(html_content, 'html.parser')
  pretty_html = soup.prettify()

  headers = soup.find_all('th')  # establishing headers
  columnheaders = []
  for i in headers:
   columnheaders.append(i.text)
  columnheaders = columnheaders[13:50]
  new_season_df = pd.DataFrame(columns = columnheaders)  # setting table based on webpage table columns

  rows = soup.find_all('tr')  # inputting the seasonal team data
  for row in rows:
    cells = row.find_all('td')
    if cells == []:
      continue
    columndata = [col.text.strip() for col in cells]
    if "NCAA" in columndata[0]:
      new_season_df.loc[len(new_season_df)]=columndata  # extracting everyone who made it into the NCAA tournament.
  new_season_df.drop(columns=['W','L'], inplace=True)  # standardizing data in ratios for model's sake
  columns_to_drop = new_season_df.columns[5:9]
  new_season_df.drop(columns=columns_to_drop,inplace=True)
  new_season_df.drop(columns=['MP','FG','FGA','FT','FTA','3P','3PA'],inplace=True)
  new_season_df = new_season_df.replace('', np.nan)
  new_season_df.iloc[:,1:] = new_season_df.iloc[:,1:].astype(float)
  column_ops = ['Tm.','Opp.','ORB','TRB','AST','STL','BLK','TOV','PF']
  for col in column_ops:
    new_season_df[col] = new_season_df[col].astype(float)
    new_season_df[col] = new_season_df[col]/(new_season_df['G'].astype(float))  # setting up per game ratios

  new_season_df.rename(columns={'Tm.':'PPG',
                               'Opp.':'PAPG',
                               'ORB':'ORBPG',
                               'TRB':'TRBPG',
                               'AST':'ASTPG',
                               'STL':'STLPG',
                               'BLK':'BLKPG',
                               'TOV':'TOVPG',
                               'PF':'PFPG'}, inplace=True)
  new_season_df.drop(columns=['G','W-L%'],inplace=True)

  new_season_df['School'] = new_season_df['School'].str.strip().str.replace('NCAA$', '', regex=True)  # changing/standardizing school names.
  new_season_df['School'] = new_season_df['School'].str.strip().str.replace('Brigham Young$', 'BYU', regex=True)
  new_season_df['School'] = new_season_df['School'].str.strip().str.replace('Connecticut$', 'UConn', regex=True)
  new_season_df['School'] = new_season_df['School'].str.strip().str.replace('North Carolina$', 'UNC', regex=True)
  new_season_df['School'] = new_season_df['School'].str.strip().str.replace("Saint Mary's (CA)", "Saint Mary's", regex=False)
  new_season_df['School'] = new_season_df['School'].str.strip().str.replace("Saint Peter's", "St. Peter's", regex=False)
  new_season_df['Year'] = url_name[49:53]
  for col in new_season_df.columns:
    if col == 'Year':
      new_season_df[col] = new_season_df[col].astype(int)
    else:
      try:
        new_season_df[col] = new_season_df[col].astype(float)
      except:
        continue
  print(f"No errors with {url_name[49:53]}")
  return new_season_df


Whew. Let's see if this works.

In [28]:
x = get_season_stats('https://www.sports-reference.com/cbb/seasons/men/2000-school-stats.html')


No errors with 2000


Nice! Now it's time to iterate and retrieve seasonal data up to 2000.

In [29]:
team_data = pd.DataFrame(columns=['School','SRS','SOS','PPG','PAPG','FG%','3P%','FT%','ORBPG','TRBPG','ASTPG','STLPG','BLKPG','TOVPG','PFPG','Year'])
for i in range(0,25):
  if i < 10:
    i = f'0{i}'
    x = get_season_stats(f'https://www.sports-reference.com/cbb/seasons/men/20{i}-school-stats.html')
  else:
    x = get_season_stats(f'https://www.sports-reference.com/cbb/seasons/men/20{i}-school-stats.html')
  team_data = pd.concat([team_data,x])
team_data.head()

No errors with 2000


<ipython-input-29-161b7a7143e7>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_data = pd.concat([team_data,x])


No errors with 2001
No errors with 2002
No errors with 2003
No errors with 2004
No errors with 2005
No errors with 2006
No errors with 2007
No errors with 2008
No errors with 2009
No errors with 2010
No errors with 2011
No errors with 2012
No errors with 2013
No errors with 2014
No errors with 2015
No errors with 2016
No errors with 2017
No errors with 2018
No errors with 2019
No errors with 2020
No errors with 2021
No errors with 2022
No errors with 2023
No errors with 2024


,School,SRS,SOS,PPG,PAPG,FG%,3P%,FT%,ORBPG,TRBPG,ASTPG,STLPG,BLKPG,TOVPG,PFPG,Year
0,Appalachian State,2.49,-3.75,79.062500,69.625000,0.486,0.388,0.709,11.687500,35.968750,16.687500,10.156250,3.968750,15.812500,19.593750,2000
1,Arizona,18.96,9.70,76.441176,67.176471,0.457,0.322,0.730,12.117647,38.411765,15.794118,7.500000,5.647059,15.000000,14.588235,2000
2,Arkansas,12.37,7.76,74.382353,69.764706,0.429,0.351,0.603,13.264706,34.088235,13.470588,11.147059,3.529412,14.529412,20.735294,2000
3,Auburn,13.59,8.20,71.264706,64.352941,0.413,0.327,0.655,15.676471,39.264706,12.882353,7.264706,3.235294,13.235294,17.117647,2000
4,Ball State,7.84,3.17,74.193548,68.967742,0.453,0.400,0.612,12.838710,36.709677,13.806452,8.322581,4.225806,13.548387,17.096774,2000


In [30]:
team_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1598 entries, 0 to 67
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   School  1598 non-null   object 
 1   SRS     1598 non-null   float64
 2   SOS     1598 non-null   float64
 3   PPG     1598 non-null   float64
 4   PAPG    1598 non-null   float64
 5   FG%     1598 non-null   float64
 6   3P%     1598 non-null   float64
 7   FT%     1598 non-null   float64
 8   ORBPG   1595 non-null   float64
 9   TRBPG   1598 non-null   float64
 10  ASTPG   1598 non-null   float64
 11  STLPG   1598 non-null   float64
 12  BLKPG   1598 non-null   float64
 13  TOVPG   1597 non-null   float64
 14  PFPG    1598 non-null   float64
 15  Year    1598 non-null   object 
dtypes: float64(14), object(2)
memory usage: 276.8+ KB


Yay! There are a couple null values, but nothing that can't be handled. Now to do the same thing for the game data.

In [53]:
tourney_df = pd.DataFrame(columns=['T1 Seed','T1 Name','T1 Score','T2 Seed','T2 Name','T2 Score','Round','Year'])
def get_game_data(url_name):
  test = requests.get(url_name)
  html_content = test.text
  soup = BeautifulSoup(html_content, 'html.parser')
  comments = soup.find_all(string=lambda text: isinstance(text, Comment))
  i = 0
  for comment in comments:
    if "game" in comment:  # the html had <--game--> comments wherever they placed bracket games.
        next_element = comment.find_next_sibling()
        try:
          seed1 = next_element.find('span').text.strip()
        except:
          break
        name1 = next_element.find('a', href=True).text.strip()
        try:
          score1 = next_element.find_all('a',href=True)[1].text.strip()
        except:
          continue
        third_element = next_element.find_next_sibling()
        seed2 = third_element.find('span').text.strip()
        name2 = third_element.find('a', href=True).text.strip()
        score2 = third_element.find_all('a',href=True)[1].text.strip()
        i += 1
        if i < 33:
          round = 1
        elif i >= 33 and i < 49:
          round = 2
        elif i >= 49 and i < 57:
          round = 3
        elif i >= 57 and i < 61:
          round = 4
        elif i >= 61 and i < 63:
          round = 5
        elif i >= 63:
          round = 6
        elif i == 63:
          break
        tourney_df.loc[len(tourney_df)] = [seed1,name1,score1,seed2,name2,score2,round,url_name[52:56]]
  print(f"No issues with {url_name[52:56]}")
  return tourney_df

In [57]:
for i in range(0,25):
  if i < 10:
    i = f'0{i}'
    x = get_game_data(f'https://www.sports-reference.com/cbb/postseason/men/20{i}-ncaa.html')
  elif i == 20:
    continue  # something came up that year
  else:
    x = get_game_data(f'https://www.sports-reference.com/cbb/postseason/men/20{i}-ncaa.html')
    tourney_df = pd.concat([tourney_df,x])


KeyboardInterrupt: 

In [56]:
tourney_df.head()

,T1 Seed,T1 Name,T1 Score,T2 Seed,T2 Name,T2 Score,Round,Year
0,1,Duke,82,16,Lamar,55,1,2000
1,8,Kansas,81,9,DePaul,77,1,2000
2,5,Florida,69,12,Butler,68,1,2000
3,4,Illinois,68,13,Penn,58,1,2000
4,6,Indiana,57,11,Pepperdine,77,1,2000
